#### Load

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

In [2]:
train=pd.read_csv('https://raw.githubusercontent.com/Chachaeul/TeamProject/master/open%20data/train.csv',index_col=0)
test=pd.read_csv('https://raw.githubusercontent.com/Chachaeul/TeamProject/master/open%20data/test_x.csv',index_col=0)
submission=pd.read_csv('https://raw.githubusercontent.com/Chachaeul/TeamProject/master/open%20data/sample_submission.csv')

In [3]:
train=train.reset_index(drop=True)
test=test.reset_index(drop=True)
train.head(30)

,QaA,QaE,QbA,QbE,QcA,QcE,QdA,QdE,QeA,QeE,...,wr_04,wr_05,wr_06,wr_07,wr_08,wr_09,wr_10,wr_11,wr_12,wr_13
0,3.0,363,4.0,1370,5.0,997,1.0,1024,2.0,1577,...,0,1,0,1,1,0,1,0,1,1
1,5.0,647,5.0,1313,3.0,3387,5.0,2969,1.0,4320,...,1,1,0,1,1,0,1,0,1,1
2,4.0,1623,1.0,1480,1.0,1021,4.0,3374,5.0,1333,...,1,1,0,1,1,1,1,0,1,1
3,3.0,504,3.0,2311,4.0,992,3.0,3245,1.0,357,...,0,0,0,0,1,0,1,0,1,1
4,1.0,927,1.0,707,5.0,556,2.0,1062,1.0,1014,...,1,1,1,1,1,0,1,1,1,1
5,2.0,834,1.0,1769,4.0,2105,1.0,1070,5.0,1930,...,0,0,0,0,1,0,0,0,0,0
6,1.0,1382,1.0,1473,5.0,1479,4.0,2403,1.0,616,...,1,1,1,1,1,0,1,0,1,1
7,1.0,384,1.0,908,5.0,870,1.0,1059,1.0,1266,...,1,1,1,1,1,1,1,0,1,1
8,5.0,795,2.0,3469,4.0,1693,3.0,1991,4.0,5922,...,0,0,0,0,1,0,0,0,1,1
9,2.0,1668,1.0,866,1.0,895,1.0,1308,2.0,1715,...,1,1,0,1,1,0,1,1,1,1


#### 특징

1. 동의하지않음 비율 ( age_group, education, engnat) 
2. Question Time 적게는 1분에서 많게는 30분인 질문들 즉, 시간이 너무 많이 쓴 데이터 제거. (사람 평균 200 밀리초) [link](https://news.joins.com/article/2869262) 출처 : [link](https://openpsychometrics.org/)
3. 종교 (0,1,11),(2,7,9,10),(3,4,5,6,8)
4. tp - 0~7 수준 나누기 01 2345 67
5. 교육수준 0,1,2 묶기 3,4 묶기
6. 인종 아랍,아시아 묶기 블랙 호주 아메리칸 아더 묶기 와이트
7. 결혼 했냐 안했냐로 0,1 2,3 묶기 
8. 나이 10대 20,30 대 40,50 대 60,70대 
9. drop ( engnat familysize gender hand urban )

#### time

In [4]:
cols=[col for col in train.columns if 'E' in col]
cols

['QaE',
 'QbE',
 'QcE',
 'QdE',
 'QeE',
 'QfE',
 'QgE',
 'QhE',
 'QiE',
 'QjE',
 'QkE',
 'QlE',
 'QmE',
 'QnE',
 'QoE',
 'QpE',
 'QqE',
 'QrE',
 'QsE',
 'QtE']

In [5]:
train[['QaE',
 'QbE',
 'QcE',
 'QdE',
 'QeE',
 'QfE',
 'QgE',
 'QhE',
 'QiE',
 'QjE',
 'QkE',
 'QlE',
 'QmE',
 'QnE',
 'QoE',
 'QpE',
 'QqE',
 'QrE',
 'QsE',
 'QtE']]

,QaE,QbE,QcE,QdE,QeE,QfE,QgE,QhE,QiE,QjE,QkE,QlE,QmE,QnE,QoE,QpE,QqE,QrE,QsE,QtE
0,363,1370,997,1024,1577,539,586,1095,1142,1287,883,851,851,816,579,924,366,876,633,1115
1,647,1313,3387,2969,4320,2190,826,4082,1867,1264,2943,3927,4329,1828,1214,2414,1356,3039,4304,1346
2,1623,1480,1021,3374,1333,531,1167,1016,2653,1569,998,2547,918,2153,1304,1131,937,1327,1170,1409
3,504,2311,992,3245,357,1519,159,2275,2809,5614,3219,1296,9046,1216,1169,23868,581,8830,2392,1312
4,927,707,556,1062,1014,628,991,1259,1153,1388,740,1181,547,575,754,1140,323,1070,583,1889
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
45527,1050,619,328,285,602,267,315,483,1016,278,611,215,388,299,1915,439,823,309,586,2252
45528,581,1353,1164,798,1680,560,640,1415,4494,1392,2478,1268,843,1401,1524,719,754,1118,654,1197
45529,593,857,1047,1515,1690,1253,1094,1283,2209,1764,1062,1489,1908,891,1298,1200,473,1779,1355,667
45530,747,1331,892,1281,1328,599,650,1429,1748,770,1025,742,4180,707,489,913,1301,1680,737,1124


In [6]:
def removeOutlier(df):
    for col in cols:
        sub=df[col]
        lower=sub.quantile(0.001)
        print(lower)
        upper=sub.quantile(0.999)
        sub=sub[(sub>=lower)&(sub<=upper)]
        df[col]=sub
    return df

train=removeOutlier(train)
train=train.dropna()
train.head(30)
train[['QaE',
 'QbE',
 'QcE',
 'QdE',
 'QeE',
 'QfE',
 'QgE',
 'QhE',
 'QiE',
 'QjE',
 'QkE',
 'QlE',
 'QmE',
 'QnE',
 'QoE',
 'QpE',
 'QqE',
 'QrE',
 'QsE',
 'QtE']]

51.062
54.531
47.0
53.062
53.0
50.0
52.531
51.531
53.062
52.0
49.531
65.124
49.531
47.0
51.531
64.531
59.0
50.0
50.531
54.531


,QaE,QbE,QcE,QdE,QeE,QfE,QgE,QhE,QiE,QjE,QkE,QlE,QmE,QnE,QoE,QpE,QqE,QrE,QsE,QtE
0,363.0,1370.0,997.0,1024.0,1577.0,539.0,586.0,1095.0,1142.0,1287.0,883.0,851.0,851.0,816.0,579.0,924.0,366.0,876.0,633.0,1115.0
1,647.0,1313.0,3387.0,2969.0,4320.0,2190.0,826.0,4082.0,1867.0,1264.0,2943.0,3927.0,4329.0,1828.0,1214.0,2414.0,1356.0,3039.0,4304.0,1346.0
2,1623.0,1480.0,1021.0,3374.0,1333.0,531.0,1167.0,1016.0,2653.0,1569.0,998.0,2547.0,918.0,2153.0,1304.0,1131.0,937.0,1327.0,1170.0,1409.0
3,504.0,2311.0,992.0,3245.0,357.0,1519.0,159.0,2275.0,2809.0,5614.0,3219.0,1296.0,9046.0,1216.0,1169.0,23868.0,581.0,8830.0,2392.0,1312.0
4,927.0,707.0,556.0,1062.0,1014.0,628.0,991.0,1259.0,1153.0,1388.0,740.0,1181.0,547.0,575.0,754.0,1140.0,323.0,1070.0,583.0,1889.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
45527,1050.0,619.0,328.0,285.0,602.0,267.0,315.0,483.0,1016.0,278.0,611.0,215.0,388.0,299.0,1915.0,439.0,823.0,309.0,586.0,2252.0
45528,581.0,1353.0,1164.0,798.0,1680.0,560.0,640.0,1415.0,4494.0,1392.0,2478.0,1268.0,843.0,1401.0,1524.0,719.0,754.0,1118.0,654.0,1197.0
45529,593.0,857.0,1047.0,1515.0,1690.0,1253.0,1094.0,1283.0,2209.0,1764.0,1062.0,1489.0,1908.0,891.0,1298.0,1200.0,473.0,1779.0,1355.0,667.0
45530,747.0,1331.0,892.0,1281.0,1328.0,599.0,650.0,1429.0,1748.0,770.0,1025.0,742.0,4180.0,707.0,489.0,913.0,1301.0,1680.0,737.0,1124.0


#### religion

In [7]:
religionMap={
    'Agnostic':0,
    'Atheist':0,
    'Other':0,
    'Buddhist':1,
    'Hindu':1,
    'Muslim':1,
    'Sikh':1,
    'Christian_Catholic':2,
    'Christian_Mormon':2,
    'Christian_Protestant':2,
    'Christian_Other':2,
    'Jewish':2
}

train['religion']=train['religion'].map(religionMap)
test['religion']=test['religion'].map(religionMap)
train.head(30)

,QaA,QaE,QbA,QbE,QcA,QcE,QdA,QdE,QeA,QeE,...,wr_04,wr_05,wr_06,wr_07,wr_08,wr_09,wr_10,wr_11,wr_12,wr_13
0,3.0,363.0,4.0,1370.0,5.0,997.0,1.0,1024.0,2.0,1577.0,...,0,1,0,1,1,0,1,0,1,1
1,5.0,647.0,5.0,1313.0,3.0,3387.0,5.0,2969.0,1.0,4320.0,...,1,1,0,1,1,0,1,0,1,1
2,4.0,1623.0,1.0,1480.0,1.0,1021.0,4.0,3374.0,5.0,1333.0,...,1,1,0,1,1,1,1,0,1,1
3,3.0,504.0,3.0,2311.0,4.0,992.0,3.0,3245.0,1.0,357.0,...,0,0,0,0,1,0,1,0,1,1
4,1.0,927.0,1.0,707.0,5.0,556.0,2.0,1062.0,1.0,1014.0,...,1,1,1,1,1,0,1,1,1,1
5,2.0,834.0,1.0,1769.0,4.0,2105.0,1.0,1070.0,5.0,1930.0,...,0,0,0,0,1,0,0,0,0,0
6,1.0,1382.0,1.0,1473.0,5.0,1479.0,4.0,2403.0,1.0,616.0,...,1,1,1,1,1,0,1,0,1,1
7,1.0,384.0,1.0,908.0,5.0,870.0,1.0,1059.0,1.0,1266.0,...,1,1,1,1,1,1,1,0,1,1
8,5.0,795.0,2.0,3469.0,4.0,1693.0,3.0,1991.0,4.0,5922.0,...,0,0,0,0,1,0,0,0,1,1
9,2.0,1668.0,1.0,866.0,1.0,895.0,1.0,1308.0,2.0,1715.0,...,1,1,0,1,1,0,1,1,1,1


#### education

In [8]:
educationMap={
    0:0,
    1:0,
    2:0,
    3:1,
    4:1
}

train['education']=train['education'].map(educationMap)
test['education']=test['education'].map(educationMap)

#### race

In [9]:
raceMap={
    'Asian':2,
    'Arab':2,
    'Black':1,
    'Indigenous Australian':1,
    'Native American':1,
    'Other':1,
    'White':0
}

train['race']=train['race'].map(raceMap)
test['race']=test['race'].map(raceMap)
train.head(30)

,QaA,QaE,QbA,QbE,QcA,QcE,QdA,QdE,QeA,QeE,...,wr_04,wr_05,wr_06,wr_07,wr_08,wr_09,wr_10,wr_11,wr_12,wr_13
0,3.0,363.0,4.0,1370.0,5.0,997.0,1.0,1024.0,2.0,1577.0,...,0,1,0,1,1,0,1,0,1,1
1,5.0,647.0,5.0,1313.0,3.0,3387.0,5.0,2969.0,1.0,4320.0,...,1,1,0,1,1,0,1,0,1,1
2,4.0,1623.0,1.0,1480.0,1.0,1021.0,4.0,3374.0,5.0,1333.0,...,1,1,0,1,1,1,1,0,1,1
3,3.0,504.0,3.0,2311.0,4.0,992.0,3.0,3245.0,1.0,357.0,...,0,0,0,0,1,0,1,0,1,1
4,1.0,927.0,1.0,707.0,5.0,556.0,2.0,1062.0,1.0,1014.0,...,1,1,1,1,1,0,1,1,1,1
5,2.0,834.0,1.0,1769.0,4.0,2105.0,1.0,1070.0,5.0,1930.0,...,0,0,0,0,1,0,0,0,0,0
6,1.0,1382.0,1.0,1473.0,5.0,1479.0,4.0,2403.0,1.0,616.0,...,1,1,1,1,1,0,1,0,1,1
7,1.0,384.0,1.0,908.0,5.0,870.0,1.0,1059.0,1.0,1266.0,...,1,1,1,1,1,1,1,0,1,1
8,5.0,795.0,2.0,3469.0,4.0,1693.0,3.0,1991.0,4.0,5922.0,...,0,0,0,0,1,0,0,0,1,1
9,2.0,1668.0,1.0,866.0,1.0,895.0,1.0,1308.0,2.0,1715.0,...,1,1,0,1,1,0,1,1,1,1


#### married

In [10]:
marriedMap={
    0:0,
    1:0,
    2:1,
    3:1
}

train['married']=train['married'].map(marriedMap)
test['married']=test['married'].map(marriedMap)
train.head(30)

,QaA,QaE,QbA,QbE,QcA,QcE,QdA,QdE,QeA,QeE,...,wr_04,wr_05,wr_06,wr_07,wr_08,wr_09,wr_10,wr_11,wr_12,wr_13
0,3.0,363.0,4.0,1370.0,5.0,997.0,1.0,1024.0,2.0,1577.0,...,0,1,0,1,1,0,1,0,1,1
1,5.0,647.0,5.0,1313.0,3.0,3387.0,5.0,2969.0,1.0,4320.0,...,1,1,0,1,1,0,1,0,1,1
2,4.0,1623.0,1.0,1480.0,1.0,1021.0,4.0,3374.0,5.0,1333.0,...,1,1,0,1,1,1,1,0,1,1
3,3.0,504.0,3.0,2311.0,4.0,992.0,3.0,3245.0,1.0,357.0,...,0,0,0,0,1,0,1,0,1,1
4,1.0,927.0,1.0,707.0,5.0,556.0,2.0,1062.0,1.0,1014.0,...,1,1,1,1,1,0,1,1,1,1
5,2.0,834.0,1.0,1769.0,4.0,2105.0,1.0,1070.0,5.0,1930.0,...,0,0,0,0,1,0,0,0,0,0
6,1.0,1382.0,1.0,1473.0,5.0,1479.0,4.0,2403.0,1.0,616.0,...,1,1,1,1,1,0,1,0,1,1
7,1.0,384.0,1.0,908.0,5.0,870.0,1.0,1059.0,1.0,1266.0,...,1,1,1,1,1,1,1,0,1,1
8,5.0,795.0,2.0,3469.0,4.0,1693.0,3.0,1991.0,4.0,5922.0,...,0,0,0,0,1,0,0,0,1,1
9,2.0,1668.0,1.0,866.0,1.0,895.0,1.0,1308.0,2.0,1715.0,...,1,1,0,1,1,0,1,1,1,1


#### age

In [11]:
def ageBand(x):
    if x=='10s':
        return 0
    elif (x=='20s')or(x=='30s'):
        return 1
    elif (x=='40s')or(x=='50s'):
        return 2
    elif (x=='60s')or(x=='+70s'):
        return 3
    
train['age_group']=train['age_group'].apply(ageBand)
test['age_group']=test['age_group'].apply(ageBand)
train.head(30)

,QaA,QaE,QbA,QbE,QcA,QcE,QdA,QdE,QeA,QeE,...,wr_04,wr_05,wr_06,wr_07,wr_08,wr_09,wr_10,wr_11,wr_12,wr_13
0,3.0,363.0,4.0,1370.0,5.0,997.0,1.0,1024.0,2.0,1577.0,...,0,1,0,1,1,0,1,0,1,1
1,5.0,647.0,5.0,1313.0,3.0,3387.0,5.0,2969.0,1.0,4320.0,...,1,1,0,1,1,0,1,0,1,1
2,4.0,1623.0,1.0,1480.0,1.0,1021.0,4.0,3374.0,5.0,1333.0,...,1,1,0,1,1,1,1,0,1,1
3,3.0,504.0,3.0,2311.0,4.0,992.0,3.0,3245.0,1.0,357.0,...,0,0,0,0,1,0,1,0,1,1
4,1.0,927.0,1.0,707.0,5.0,556.0,2.0,1062.0,1.0,1014.0,...,1,1,1,1,1,0,1,1,1,1
5,2.0,834.0,1.0,1769.0,4.0,2105.0,1.0,1070.0,5.0,1930.0,...,0,0,0,0,1,0,0,0,0,0
6,1.0,1382.0,1.0,1473.0,5.0,1479.0,4.0,2403.0,1.0,616.0,...,1,1,1,1,1,0,1,0,1,1
7,1.0,384.0,1.0,908.0,5.0,870.0,1.0,1059.0,1.0,1266.0,...,1,1,1,1,1,1,1,0,1,1
8,5.0,795.0,2.0,3469.0,4.0,1693.0,3.0,1991.0,4.0,5922.0,...,0,0,0,0,1,0,0,0,1,1
9,2.0,1668.0,1.0,866.0,1.0,895.0,1.0,1308.0,2.0,1715.0,...,1,1,0,1,1,0,1,1,1,1


#### tp

In [12]:
def Filter(x):
    if x<1:
        return 0
    elif x<2:
        return 0
    elif x<6:
        return 1
    else:
        return 2
    
cols=[col for col in train.columns if 'tp' in col]

def get_tp(df):
    for col in cols:
        df[col]=df[col].apply(Filter)
    return df

train=get_tp(train)
test=get_tp(test)
train.head(30)

,QaA,QaE,QbA,QbE,QcA,QcE,QdA,QdE,QeA,QeE,...,wr_04,wr_05,wr_06,wr_07,wr_08,wr_09,wr_10,wr_11,wr_12,wr_13
0,3.0,363.0,4.0,1370.0,5.0,997.0,1.0,1024.0,2.0,1577.0,...,0,1,0,1,1,0,1,0,1,1
1,5.0,647.0,5.0,1313.0,3.0,3387.0,5.0,2969.0,1.0,4320.0,...,1,1,0,1,1,0,1,0,1,1
2,4.0,1623.0,1.0,1480.0,1.0,1021.0,4.0,3374.0,5.0,1333.0,...,1,1,0,1,1,1,1,0,1,1
3,3.0,504.0,3.0,2311.0,4.0,992.0,3.0,3245.0,1.0,357.0,...,0,0,0,0,1,0,1,0,1,1
4,1.0,927.0,1.0,707.0,5.0,556.0,2.0,1062.0,1.0,1014.0,...,1,1,1,1,1,0,1,1,1,1
5,2.0,834.0,1.0,1769.0,4.0,2105.0,1.0,1070.0,5.0,1930.0,...,0,0,0,0,1,0,0,0,0,0
6,1.0,1382.0,1.0,1473.0,5.0,1479.0,4.0,2403.0,1.0,616.0,...,1,1,1,1,1,0,1,0,1,1
7,1.0,384.0,1.0,908.0,5.0,870.0,1.0,1059.0,1.0,1266.0,...,1,1,1,1,1,1,1,0,1,1
8,5.0,795.0,2.0,3469.0,4.0,1693.0,3.0,1991.0,4.0,5922.0,...,0,0,0,0,1,0,0,0,1,1
9,2.0,1668.0,1.0,866.0,1.0,895.0,1.0,1308.0,2.0,1715.0,...,1,1,0,1,1,0,1,1,1,1


#### question

In [13]:
group45=['QaA','QqA']
group1=['QbA','QcA','QoA','QpA','QsA','QtA']
group12=['QjA','QmA']

cols=[col for col in train.columns if 'A' in col]
dropCol=[]
def question(df):
    for col in cols:
        if col in group1:
            df[col]=df[col].map({1:1,2:0,3:0,4:0,5:0})
        elif col in group12:
            df[col]=df[col].map({1:1,2:1,3:0,4:0,5:0})
        elif col in group45:
            df[col]=df[col].map({1:0,2:0,3:0,4:1,5:1})
        else:
            dropCol.append(col)
    return df

train=question(train)
test=question(test)
train.head(30)

,QaA,QaE,QbA,QbE,QcA,QcE,QdA,QdE,QeA,QeE,...,wr_04,wr_05,wr_06,wr_07,wr_08,wr_09,wr_10,wr_11,wr_12,wr_13
0,0,363.0,0,1370.0,0,997.0,1.0,1024.0,2.0,1577.0,...,0,1,0,1,1,0,1,0,1,1
1,1,647.0,0,1313.0,0,3387.0,5.0,2969.0,1.0,4320.0,...,1,1,0,1,1,0,1,0,1,1
2,1,1623.0,1,1480.0,1,1021.0,4.0,3374.0,5.0,1333.0,...,1,1,0,1,1,1,1,0,1,1
3,0,504.0,0,2311.0,0,992.0,3.0,3245.0,1.0,357.0,...,0,0,0,0,1,0,1,0,1,1
4,0,927.0,1,707.0,0,556.0,2.0,1062.0,1.0,1014.0,...,1,1,1,1,1,0,1,1,1,1
5,0,834.0,1,1769.0,0,2105.0,1.0,1070.0,5.0,1930.0,...,0,0,0,0,1,0,0,0,0,0
6,0,1382.0,1,1473.0,0,1479.0,4.0,2403.0,1.0,616.0,...,1,1,1,1,1,0,1,0,1,1
7,0,384.0,1,908.0,0,870.0,1.0,1059.0,1.0,1266.0,...,1,1,1,1,1,1,1,0,1,1
8,1,795.0,0,3469.0,0,1693.0,3.0,1991.0,4.0,5922.0,...,0,0,0,0,1,0,0,0,1,1
9,0,1668.0,1,866.0,1,895.0,1.0,1308.0,2.0,1715.0,...,1,1,0,1,1,0,1,1,1,1


#### not used cols

In [14]:
not_usecols=['engnat','familysize','gender','hand','urban',
            'wf_01','wf_02','wf_03','wr_01','wr_02','wr_03',
             'wr_04','wr_05','wr_06','wr_07','wr_08','wr_09',
             'wr_10','wr_11','wr_12','wr_13']

def change(df):
    for col in not_usecols:
        df=df.drop(col,axis=1)
    return df

train=change(train)
test=change(test)
train.head(30)

,QaA,QaE,QbA,QbE,QcA,QcE,QdA,QdE,QeA,QeE,...,tp02,tp03,tp04,tp05,tp06,tp07,tp08,tp09,tp10,voted
0,0,363.0,0,1370.0,0,997.0,1.0,1024.0,2.0,1577.0,...,1,1,0,1,0,2,1,1,1,2
1,1,647.0,0,1313.0,0,3387.0,5.0,2969.0,1.0,4320.0,...,0,0,0,0,1,1,1,0,1,2
2,1,1623.0,1,1480.0,1,1021.0,4.0,3374.0,5.0,1333.0,...,1,0,1,1,1,1,2,0,1,1
3,0,504.0,0,2311.0,0,992.0,3.0,3245.0,1.0,357.0,...,1,0,0,0,1,0,1,0,1,1
4,0,927.0,1,707.0,0,556.0,2.0,1062.0,1.0,1014.0,...,0,0,2,0,1,0,2,1,2,1
5,0,834.0,1,1769.0,0,2105.0,1.0,1070.0,5.0,1930.0,...,1,1,1,1,1,1,2,1,1,2
6,0,1382.0,1,1473.0,0,1479.0,4.0,2403.0,1.0,616.0,...,1,1,1,0,0,1,1,0,1,1
7,0,384.0,1,908.0,0,870.0,1.0,1059.0,1.0,1266.0,...,2,0,0,0,0,0,0,1,2,1
8,1,795.0,0,3469.0,0,1693.0,3.0,1991.0,4.0,5922.0,...,0,1,0,1,1,1,1,1,1,1
9,0,1668.0,1,866.0,1,895.0,1.0,1308.0,2.0,1715.0,...,0,0,1,0,1,0,1,0,1,1


#### mean encoding

In [15]:
from sklearn.model_selection import StratifiedKFold

kfold=StratifiedKFold(n_splits=5,shuffle=True,random_state=2020)
cols=['age_group', 'education', 'married', 'race', 'religion']
globalMean=train['voted'].mean()
alpha=5
encode_=pd.DataFrame()

for col in cols:
    parts=[]
    for train_idx,valid_idx in kfold.split(train.drop('voted',axis=1),train['voted']):
        X_tr,X_val=train.iloc[train_idx],train.iloc[valid_idx]
        nrows=X_tr.groupby(col)['voted'].count()
        mean=X_tr.groupby(col)['voted'].mean()
        smooth=(mean*nrows+globalMean*alpha)/(alpha+nrows)
        test[f'mean_{col}']=test[col].map(smooth)
        encoded=X_val[col].map(smooth)
        parts.append(encoded)
    encode=pd.concat(parts,axis=0)
    encode_=pd.concat([encode_,encode],axis=1)
encode_=encode_.sort_index()
encode_.columns=[f"mean_{col}" for col in encode_.columns]
train=pd.concat([train,encode_],axis=1)
train.head(30)

,QaA,QaE,QbA,QbE,QcA,QcE,QdA,QdE,QeA,QeE,...,tp07,tp08,tp09,tp10,voted,mean_age_group,mean_education,mean_married,mean_race,mean_religion
0,0,363.0,0,1370.0,0,997.0,1.0,1024.0,2.0,1577.0,...,2,1,1,1,2,1.450568,1.702243,1.359979,1.496091,1.564135
1,1,647.0,0,1313.0,0,3387.0,5.0,2969.0,1.0,4320.0,...,1,1,0,1,2,1.448313,1.393857,1.629800,1.704078,1.641271
2,1,1623.0,1,1480.0,1,1021.0,4.0,3374.0,5.0,1333.0,...,1,2,0,1,1,1.449234,1.395471,1.360093,1.497131,1.563386
3,0,504.0,0,2311.0,0,992.0,3.0,3245.0,1.0,357.0,...,0,1,0,1,1,1.448556,1.394144,1.629827,1.704341,1.630551
4,0,927.0,1,707.0,0,556.0,2.0,1062.0,1.0,1014.0,...,0,2,1,2,1,1.449234,1.395471,1.360093,1.497131,1.563386
5,0,834.0,1,1769.0,0,2105.0,1.0,1070.0,5.0,1930.0,...,1,2,1,1,2,1.348637,1.700218,1.356379,1.494978,1.562086
6,0,1382.0,1,1473.0,0,1479.0,4.0,2403.0,1.0,616.0,...,1,1,0,1,1,1.449234,1.703134,1.629322,1.497131,1.563386
7,0,384.0,1,908.0,0,870.0,1.0,1059.0,1.0,1266.0,...,0,0,1,2,1,1.449651,1.700218,1.630116,1.494978,1.505536
8,1,795.0,0,3469.0,0,1693.0,3.0,1991.0,4.0,5922.0,...,1,1,1,1,1,1.449651,1.397175,1.630116,1.616141,1.505536
9,0,1668.0,1,866.0,1,895.0,1.0,1308.0,2.0,1715.0,...,0,1,0,1,1,1.349989,1.395471,1.629322,1.497131,1.504675


In [16]:
train=train.drop(dropCol,axis=1)
test=test.drop(dropCol,axis=1)
train.head(30)

,QaA,QaE,QbA,QbE,QcA,QcE,QdE,QeE,QfE,QgE,...,tp07,tp08,tp09,tp10,voted,mean_age_group,mean_education,mean_married,mean_race,mean_religion
0,0,363.0,0,1370.0,0,997.0,1024.0,1577.0,539.0,586.0,...,2,1,1,1,2,1.450568,1.702243,1.359979,1.496091,1.564135
1,1,647.0,0,1313.0,0,3387.0,2969.0,4320.0,2190.0,826.0,...,1,1,0,1,2,1.448313,1.393857,1.629800,1.704078,1.641271
2,1,1623.0,1,1480.0,1,1021.0,3374.0,1333.0,531.0,1167.0,...,1,2,0,1,1,1.449234,1.395471,1.360093,1.497131,1.563386
3,0,504.0,0,2311.0,0,992.0,3245.0,357.0,1519.0,159.0,...,0,1,0,1,1,1.448556,1.394144,1.629827,1.704341,1.630551
4,0,927.0,1,707.0,0,556.0,1062.0,1014.0,628.0,991.0,...,0,2,1,2,1,1.449234,1.395471,1.360093,1.497131,1.563386
5,0,834.0,1,1769.0,0,2105.0,1070.0,1930.0,1258.0,1451.0,...,1,2,1,1,2,1.348637,1.700218,1.356379,1.494978,1.562086
6,0,1382.0,1,1473.0,0,1479.0,2403.0,616.0,549.0,390.0,...,1,1,0,1,1,1.449234,1.703134,1.629322,1.497131,1.563386
7,0,384.0,1,908.0,0,870.0,1059.0,1266.0,483.0,592.0,...,0,0,1,2,1,1.449651,1.700218,1.630116,1.494978,1.505536
8,1,795.0,0,3469.0,0,1693.0,1991.0,5922.0,1322.0,1014.0,...,1,1,1,1,1,1.449651,1.397175,1.630116,1.616141,1.505536
9,0,1668.0,1,866.0,1,895.0,1308.0,1715.0,563.0,574.0,...,0,1,0,1,1,1.349989,1.395471,1.629322,1.497131,1.504675


#### frequent encoding

In [25]:
cols=['age_group','education','married','race','religion']

for col in cols:
    freqMap=train.groupby(col)['voted'].count()/train.shape[0]
    
    train[f'freq_{col}']=train[col].map(freqMap)
    test[f'freq_{col}']=test[col].map(freqMap)
    
train.head(30)
abs(train.corr()[['voted']]).sort_values(by = 'voted', ascending = False).head(10)

,voted
voted,1.000000
mean_age_group,0.402593
age_group,0.367847
freq_education,0.308806
education,0.308806
mean_education,0.308691
married,0.251201
freq_married,0.251201
mean_married,0.251152
mean_race,0.160228


#### split X,y

In [18]:
train['voted']=train['voted'].map({2:1,1:0})
train.head(30)
# test.head(30)

,QaA,QaE,QbA,QbE,QcA,QcE,QdE,QeE,QfE,QgE,...,mean_age_group,mean_education,mean_married,mean_race,mean_religion,freq_age_group,freq_education,freq_married,freq_race,freq_religion
0,0,363.0,0,1370.0,0,997.0,1024.0,1577.0,539.0,586.0,...,1.450568,1.702243,1.359979,1.496091,1.564135,0.481946,0.492565,0.305617,0.688106,0.540213
1,1,647.0,0,1313.0,0,3387.0,2969.0,4320.0,2190.0,826.0,...,1.448313,1.393857,1.629800,1.704078,1.641271,0.481946,0.507435,0.694383,0.156599,0.078378
2,1,1623.0,1,1480.0,1,1021.0,3374.0,1333.0,531.0,1167.0,...,1.449234,1.395471,1.360093,1.497131,1.563386,0.481946,0.507435,0.305617,0.688106,0.540213
3,0,504.0,0,2311.0,0,992.0,3245.0,357.0,1519.0,159.0,...,1.448556,1.394144,1.629827,1.704341,1.630551,0.481946,0.507435,0.694383,0.156599,0.078378
4,0,927.0,1,707.0,0,556.0,1062.0,1014.0,628.0,991.0,...,1.449234,1.395471,1.360093,1.497131,1.563386,0.481946,0.507435,0.305617,0.688106,0.540213
5,0,834.0,1,1769.0,0,2105.0,1070.0,1930.0,1258.0,1451.0,...,1.348637,1.700218,1.356379,1.494978,1.562086,0.175317,0.492565,0.305617,0.688106,0.540213
6,0,1382.0,1,1473.0,0,1479.0,2403.0,616.0,549.0,390.0,...,1.449234,1.703134,1.629322,1.497131,1.563386,0.481946,0.492565,0.694383,0.688106,0.540213
7,0,384.0,1,908.0,0,870.0,1059.0,1266.0,483.0,592.0,...,1.449651,1.700218,1.630116,1.494978,1.505536,0.481946,0.492565,0.694383,0.688106,0.381409
8,1,795.0,0,3469.0,0,1693.0,1991.0,5922.0,1322.0,1014.0,...,1.449651,1.397175,1.630116,1.616141,1.505536,0.481946,0.507435,0.694383,0.155295,0.381409
9,0,1668.0,1,866.0,1,895.0,1308.0,1715.0,563.0,574.0,...,1.349989,1.395471,1.629322,1.497131,1.504675,0.175317,0.507435,0.694383,0.688106,0.381409


In [19]:
X=train.drop('voted',axis=1)
y=train['voted']

In [20]:
import lightgbm
from sklearn.model_selection import StratifiedKFold

cv=StratifiedKFold(n_splits=5,shuffle=True,random_state=2020)

preds=[]

params={
    'objective':'binary',
    'metric':'binary_logloss',
    'unbalance':True,
    'learning_rate':0.05,
    'verbosity':-1
}

for train_idx,valid_idx in cv.split(X,y):
    X_tr,X_val=X.iloc[train_idx],X.iloc[valid_idx]
    y_tr,y_val=y.iloc[train_idx],y.iloc[valid_idx]
    
    lgb_train=lightgbm.Dataset(X_tr,y_tr)
    lgb_valid=lightgbm.Dataset(X_val,y_val,reference=lgb_train)
    
    model=lightgbm.train(params,lgb_train,num_boost_round=2000,valid_sets=[lgb_valid],
                        early_stopping_rounds=70,verbose_eval=0)
    
    pred=model.predict(test,num_iteration=model.best_iteration)
    preds.append(pred)
print("done")

done


In [21]:
submission['voted']=np.mean(preds,axis=0)+1

In [22]:
submission.to_csv('C:\\Users\\id272\\LSJ\\Final_Project\\open data/submission_lgbm.csv',index=False)